In [6]:
import math
from openpyxl import load_workbook
from unipath import Path
import os
from django.core import exceptions

base = Path('d:\\', 'Users', 'javier', 'tmp')
r06 = Path(base, 'mac_06.xlsx')
r07 = Path(base, 'mac_07.xlsx')
r07b = Path(base, 'mac_07b.xlsx')
# wb6 = load_workbook(filename = r06)
# wb7 = load_workbook(filename = r07)
# wb7b = load_workbook(filename = r07b)
# ws6 = wb6['2021_06']
# ws7 = wb7['2021_06']
# ws7b = wb7b['2021_06']

In [7]:
def get_int(celda):
    """Convierte el valor de una celda en entero"""
    if celda.value == '':
        return 0
    try:
        if celda.ctype != 2:
            return 0
        return math.ceil(celda.value)
    except ValueError:
        return 0


def get_float(celda):
    """"Convierte el valor de una celda en un número flotante"""
    # valor = 0
    try:
        valor = float(celda.value)
    except ValueError:
        valor = 0
    return valor

In [9]:
def get_data_excel(file):
    try: 
        wb = load_workbook(filename = file, data_only=True)
    except Exception: 
        raise BadRequest(e)
    ws = wb.active
    remesa = ws['J7'].value[8:15]
    observaciones = ws['A41'].value
    macs = {}
    
    for row in range(9, 22):
        mac = ws[row]
        _mac = str(mac[0].value)
        if _mac[:3] == '290':
            macs[_mac] = {
                'distrito': _mac[2:4],
                'tipo': mac[1].value,
                'dias_trabajados': mac[2].value,
                'jornada_trabajada': mac[3].value,
                'configuracion': mac[4].value,
                'tramites_aplicados': mac[5].value,
                'credenciales_entregadas_actualizacion': mac[6].value,
                'total_atenciones': mac[7].value,
                'productividad_x_dia': mac[8].value,
                'productividad_x_estacion': mac[9].value,
                'credenciales_recibidas': mac[11].value
            }
    
    print(remesa, '\n', observaciones, '\n', macs)

get_data_excel(r06)

2021_06 
 En está remesa no se presento ninguna incidencia que suspendiera el servicio en los Módulos de Atención Ciudadana. 
 {'290151': {'distrito': '01', 'tipo': 'Fijo Dist.', 'dias_trabajados': 6.5, 'jornada_trabajada': 10.2, 'configuracion': 'B+3', 'tramites_aplicados': 1482, 'credenciales_entregadas_actualizacion': 17, 'total_atenciones': 1499, 'productividad_x_dia': 230.6153846153846, 'productividad_x_estacion': 57.65384615384615, 'credenciales_recibidas': 1304}, '290152': {'distrito': '01', 'tipo': 'Fijo Adic.', 'dias_trabajados': 6.5, 'jornada_trabajada': 10.2, 'configuracion': 'B+2', 'tramites_aplicados': 1282, 'credenciales_entregadas_actualizacion': 68, 'total_atenciones': 1350, 'productividad_x_dia': 207.69230769230768, 'productividad_x_estacion': 69.23076923076923, 'credenciales_recibidas': 357}, '290153': {'distrito': '01', 'tipo': 'Móvil', 'dias_trabajados': 6.5, 'jornada_trabajada': 6.5, 'configuracion': 'B', 'tramites_aplicados': 330, 'credenciales_entregadas_actualiz

Remesa: 2021_06
